### Importing Libraries

In [95]:
import pandas as pd
import numpy as np
import networkx as nx

from networkx import neighbors
from networkx import common_neighbors

from matplotlib.pyplot import figure

from sklearn.model_selection import train_test_split

from tqdm import tqdm

### Importing Dataset

In [96]:
df = pd.read_csv("data.tsv", sep="\t")
df

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES
0,leagueoflegends,teamredditteams,1u4nrps,2013-12-31 16:39:58,1,"345.0,298.0,0.75652173913,0.0173913043478,0.08..."
1,theredlion,soccer,1u4qkd,2013-12-31 18:18:37,-1,"101.0,98.0,0.742574257426,0.019801980198,0.049..."
2,inlandempire,bikela,1u4qlzs,2014-01-01 14:54:35,1,"85.0,85.0,0.752941176471,0.0235294117647,0.082..."
3,nfl,cfb,1u4sjvs,2013-12-31 17:37:55,1,"1124.0,949.0,0.772241992883,0.0017793594306,0...."
4,playmygame,gamedev,1u4w5ss,2014-01-01 02:51:13,1,"715.0,622.0,0.777622377622,0.00699300699301,0...."
...,...,...,...,...,...,...
286556,negareddit,debatefascism,68im20s,2017-04-30 16:31:26,1,"441.0,405.0,0.775510204082,0.0294784580499,0.0..."
286557,mildlynomil,justnomil,68imlas,2017-04-30 04:19:03,1,"2226.0,1855.0,0.786163522013,0.00224618149146,..."
286558,mmorpg,blackdesertonline,68ip5os,2017-04-30 16:54:08,1,"1100.0,909.0,0.778181818182,0.00181818181818,0..."
286559,electricskateboards,askreddit,68ipb2s,2017-04-30 16:41:53,1,"1876.0,1567.0,0.78144989339,0.00692963752665,0..."


### Creating the network structure

In [97]:
G = nx.Graph()
G = nx.from_pandas_edgelist(df, 'SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT', edge_attr='LINK_SENTIMENT')

### Data Statistics

In [116]:
leaderboard = {}
for x in G.nodes:
 leaderboard[x] = len(G[x])
s = pd.Series(leaderboard, name='connections')
df_leaderboard = s.to_frame().sort_values('connections', ascending=False)

In [117]:
df_leaderboard

,connections
askreddit,2336
iama,1839
subredditdrama,1598
writingprompts,1043
outoftheloop,986
...,...
onetruekongou,1
csfragfilms,1
debatereligioncss,1
gunnitrust,1


In [118]:
avg = df_leaderboard["connections"].mean()
print("The average number of connections of a node are " + str(avg))

The average number of connections of a node are 6.950469588550984


In [119]:
unique_links = df["POST_ID"].nunique()
print("The number of unique subreddits are " + str(unique_links))

The number of unique subreddits are 259092


#### Make a dictionary for neighbors of nodes

In [120]:
def add_values_in_dict(given_dict, key, list_of_values):
    if key not in given_dict:
        given_dict[key] = list()
    given_dict[key].extend(list_of_values)
    return

In [121]:
edge_list = {}
for x in G.nodes:
    temp_neighbour = [n for n in G.neighbors(x)]
    add_values_in_dict(edge_list, x, temp_neighbour)

### Preprocessing
#### Splitting the data into train data and test data

In [122]:
train, test = train_test_split(df, test_size=0.2)

In [123]:
train_G = nx.Graph()
train_G = nx.from_pandas_edgelist(train, 'SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT', edge_attr='LINK_SENTIMENT')
test_G = nx.Graph()
test_G = nx.from_pandas_edgelist(test, 'SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT', edge_attr='LINK_SENTIMENT')

#### Algorithm for link prediction

In [124]:
def link_pred_cn(G, m):
    new_edges = []
    e = list(G.edges())
    for i in tqdm(e):
        a, b = i
        if(G.get_edge_data(a, b)['LINK_SENTIMENT']==m):
            for j in e:
                x, y = j
                if(G.get_edge_data(x, y)['LINK_SENTIMENT']==m):
                    if i != j:
                        if a == x and (b, y) not in e and (y, b) not in e:
                            if(len(sorted(nx.common_neighbors(G, y, b)))>=30):
                                new_edges.append((b, y))
                        if a == y and (b, x) not in e and (x, b) not in e:
                            if(len(sorted(nx.common_neighbors(G, x, b)))>=30):
                                new_edges.append((b, x))
                        if b == x and (a, y) not in e and (y, a) not in e:
                            if(len(sorted(nx.common_neighbors(G, y, a)))>=30):
                                new_edges.append((a, y))
                        if b == y and (a, x) not in e and (x, a) not in e:
                            if(len(sorted(nx.common_neighbors(G, x, a)))>=30):
                                new_edges.append((a, x))
    return new_edges

### Making Predictions

In [ ]:
pred_pos_edges = link_pred_cn(train_G, 1)

  0%|          | 182/105470 [02:51<34:24:21,  1.18s/it]

In [ ]:
pred_neg_edges = link_pred_cn(train_G, -1)

In [125]:
test_edges = list(test_G.edges())
test_pos_edges = []
test_neg_edges = []
for i in tqdm(test_edges):
    x, y = i
    if(G.get_edge_data(x, y)['LINK_SENTIMENT']==1):
        test_pos_edges.append((x, y))
    if(G.get_edge_data(x, y)['LINK_SENTIMENT']==-1):
        test_pos_edges.append((x, y))

100%|██████████| 36191/36191 [00:00<00:00, 696008.88it/s]


### Finding accuracy for the algorithm

In [126]:
def pos_count(e):
    tp = 0
    fp = 0
    for i in tqdm(e):
        x, y = i
        for j in test_pos_edges:
            a, b = j
            if (a == x and b == y) or (a == y and b == x):
                tp = tp + 1
            else:
                fp = fp + 1
    return tp, fp

In [127]:
def neg_count(e):
    tn = 0
    fn = 0
    for i in tqdm(e):
        x, y = i
        for j in test_neg_edges:
            a, b = j
            if (a == x and b == y) or (a == y and b == x):
                tn = tn + 1
            else:
                fn = fn + 1
    return tn, fn

In [128]:
positive_class = pos_count(pred_pos_edges)
negative_class = neg_count(pred_neg_edges)
accuracy = (positive_class[0]+negative_class[0])/(positive_class[0]+negative_class[0]+positive_class[1]+negative_class[1])
print("The accuracy of the preiction algorithm is " + str(accuracy))

100%|██████████| 168/168 [00:00<00:00, 1334551.27it/s]

The accuracy of the preiction algorithm is 8.279092296552291e-06
